In [1]:
#Import dependencies
import pandas as pd
import numpy as np
import os

In [2]:
# Read the CSV file from EMEC
base_path= "Resources/ingresos/emec.csv"
reader_df = pd.read_csv(base_path)
reader_df.head()

,CODIGO_ACTIVIDAD,ANIO,MES,ENTIDAD,K100W,M000W,H000W_I000W,REMUNERACION_MEDIA,J000W,ESTATUS
0,43,2008,1,Aguascalientes,74.245893,78.029304,96.969659,91.295343,88.528782,Cifras definitivas
1,43,2008,1,Baja California,100.174498,103.944683,83.487349,97.787119,81.639874,Cifras definitivas
2,43,2008,1,Baja California Sur,87.453394,96.040172,80.932986,116.502902,94.289277,Cifras definitivas
3,43,2008,1,Campeche,88.239763,81.449487,91.839089,66.608787,61.172903,Cifras definitivas
4,43,2008,1,Coahuila de Zaragoza,107.501543,95.407155,90.027855,96.768532,87.118633,Cifras definitivas


In [3]:
# Get the info from 2020
columns_df = ["CODIGO_ACTIVIDAD", "ANIO","MES","ENTIDAD","M000W"]
clean_df = reader_df[(reader_df['ANIO'] == 2020)]
clean_df = clean_df[columns_df]

trial_df = clean_df[(clean_df["ENTIDAD"] == "Aguascalientes")]
trial_df

,CODIGO_ACTIVIDAD,ANIO,MES,ENTIDAD,M000W
4608,43,2020,1,Aguascalientes,140.382115
4640,43,2020,2,Aguascalientes,134.516830
4672,43,2020,3,Aguascalientes,139.511460
4704,43,2020,4,Aguascalientes,117.148580
4736,43,2020,5,Aguascalientes,120.866463
4768,43,2020,6,Aguascalientes,126.714480
4800,43,2020,7,Aguascalientes,139.756476
4832,43,2020,8,Aguascalientes,136.138315
4864,43,2020,9,Aguascalientes,132.122284
4896,43,2020,10,Aguascalientes,140.323964


In [18]:
# Add the CVE_ENT column in the dataframe

# Create the dataframe for state and its corresponding CVE_ENT
states_df = pd.DataFrame()

states  = clean_df["ENTIDAD"].unique()
state_key = list(range(1,33,1))

states_df["ENTIDAD"] = states
states_df["CVE_ENT"] = state_key
states_df

merge_df = pd.merge(clean_df, states_df, on="ENTIDAD")
merge_df

,CODIGO_ACTIVIDAD,ANIO,MES,ENTIDAD,M000W,CVE_ENT
0,43,2020,1,Aguascalientes,140.382115,1
1,43,2020,2,Aguascalientes,134.516830,1
2,43,2020,3,Aguascalientes,139.511460,1
3,43,2020,4,Aguascalientes,117.148580,1
4,43,2020,5,Aguascalientes,120.866463,1
...,...,...,...,...,...,...
699,46,2020,7,Zacatecas,121.901501,32
700,46,2020,8,Zacatecas,121.118065,32
701,46,2020,9,Zacatecas,121.151944,32
702,46,2020,10,Zacatecas,123.434419,32


In [19]:
# Group by Clave de Entidad and CODIGO_ACTIVIDAD
grp_df = merge_df.groupby(["CVE_ENT", "CODIGO_ACTIVIDAD"])\
                           .agg({"M000W": "mean"})\
                           .rename(columns = {"M000W": "INGRESO PROM MEN"})
grp_df["INGRESO ANUAL"] = grp_df["INGRESO PROM MEN"]*12

In [20]:
# Calculate the estimated earnings per type of company based on historic data
porc_micro = .10
porc_peq = .20
grp_df["0 a 5 personas"] = grp_df["INGRESO ANUAL"] * porc_micro
grp_df["6 a 10 personas"] = grp_df["INGRESO ANUAL"] * porc_peq

grp_df

INGRESO PROM MEN  INGRESO ANUAL  0 a 5 personas  \
CVE_ENT CODIGO_ACTIVIDAD                                                    
1       43                      132.819391    1593.832690      159.383269   
        46                      114.790322    1377.483864      137.748386   
2       43                      125.623340    1507.480081      150.748008   
        46                      150.177299    1802.127591      180.212759   
3       43                      114.878722    1378.544669      137.854467   
...                                    ...            ...             ...   
30      46                       99.663981    1195.967769      119.596777   
31      43                      104.497562    1253.970739      125.397074   
        46                      122.107349    1465.288183      146.528818   
32      43                      102.953664    1235.443963      123.544396   
        46                      118.078655    1416.943857      141.694386   

                          6 a 10 personas  
CVE_ENT CODIGO_ACTIVIDAD                   
1       43                     318.766538  
        46                     275.496773  
2       43                     301.496016  
        46                     360.425518  
3       43                     275.708934  
...                                   ...  
30      46                     239.193554  
31      43                     250.794148  
        46                     293.057637  
32      43                     247.088793  
        46                     283.388771  

[64 rows x 4 columns]

In [22]:
# Export the results to a CSV file
csv_path= "Output/emec_total.csv"
grp_df.to_csv(csv_path,index=True)